## EXTRACT - TRANSFORM - LOAD 

In [2]:
#import Libraries 
import requests
import pandas as pd 
from   datetime import datetime 
import sqlite3

In [12]:
# EXTRACT

# API endpoint for fetching all current rates
url = "https://rest.coinapi.io/v1/exchangerate/BTC/USD"

# Your API key
headers = {
    'X-CoinAPI-Key': '0A4EC419-E056-44E2-B22B-CEDDB3C09AC0'
}

# Send the request to get the current Bitcoin price in USD
response = requests.get(url, headers=headers)

# TRANSFORM a
# Check if the request was successful
if response.status_code == 200:
    
    # TRANSFROM
    data = response.json()
    dt = datetime.strptime(data['time'], "%Y-%m-%dT%H:%M:%S.%f0Z")

    # Load
    date = dt.date().__str__()
    print(type(date))
    dict = {
    'Date': date,
    'Price': data['rate'],
    'Hour': dt.hour,
    'Minute': dt.minute
    }
    
    print(dict)
    # Convert JSON data to a pandas DataFrame
    df = pd.DataFrame([{
        'asset_id_base':  data['asset_id_base'],
        'asset_id_quote': data['asset_id_quote'],
        'rate':   data['rate'],
        'Date':   dt.date(),
        'Hour':   dt.hour,
        'Minute': dt.minute
        }])
    
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")



<class 'str'>
{'Date': '2024-09-24', 'Price': 63034.93862111122, 'Hour': 14, 'Minute': 48}


In [7]:
df

,asset_id_base,asset_id_quote,rate,Date,Hour,Minute
0,BTC,USD,63405.934171,2024-09-20,12,38


Establish connection

find table with pecific name in connection
    if no, create table, then append data 
    if yes , append data 

In [11]:
conn = sqlite3.connect("pricedata.db")
cur = conn.cursor()

cur.execute('''
    CREATE TABLE IF NOT EXISTS price_data (
        Date    DATE,
        Price   FLOAT,
        Hour    INTEGER,
        Minute  INTEGER
    )''')

cur.execute('''
    INSERT INTO price_data (Date, Price, Hour, Minute) 
    VALUES (?, ?, ?, ?)
''', (dt.date(), data['rate'], dt.hour, dt.minute))

conn.commit()
conn.close()

print("Data successfully inserted into the database.")


Data successfully inserted into the database.
